In [ ]:
import sys
sys.path.append('../../common')
import utils
import importlib
importlib.reload(utils)

import time
import random 
import time
import copy

import numpy as np
import pynput.keyboard as kb

from enum import Enum

io = utils.IOHandler(offset=(880, 558), game_dims=(1600,1200), verbose=True)

In [ ]:
class Atom(Enum):
  BLUE = "blue"
  GREEN = "green"
  CYAN = "cyan"
  PURPLE = "purple"
  PINK = "pink"
  RED = "red"
  ORANGE = "orange"
  YELLOW = "yellow"
  BLANK = "blank"

class Style(Enum):
  STAR = "star"
  DOT = "dot"
  PLAIN = "plain"

class Screen(Enum):
  OUT_OF_MOVES = "out_of_moves"
  LEVEL_COMPLETE = "level_complete"
  OUT_OF_TIME = "out_of_time"
  HOME = "home"
  GAME_OVER = "game_over"
  IN_GAME = "in_game"
  ARCADE = "arcade"
  ARCADE_GOOBER_BOTTOM = "arcade_goober_bottom"
  ARCADE_GOOBER_SELECTED = "arcade_goober_selected"
  GREAT_PLAY = "great_play"

# TODO: move to utils?
SCREEN_ACTIONS = {
  Screen.OUT_OF_MOVES: lambda: io.click_mouse(1550, 50, delays=[0.3,0,0]), # "x" out button
  Screen.LEVEL_COMPLETE: lambda: io.click_mouse(800, 1015, delays=[0.3,0,0]),
  Screen.OUT_OF_TIME: lambda: io.click_mouse(1550, 50, delays=[0.3,0,0]), # "x" out button
  Screen.HOME: lambda: io.click_mouse(540, 940, delays=[0.3,0,0]), # "PLAY" button
  Screen.GAME_OVER: lambda: io.click_mouse(1030, 890, delays=[0.1,0,0.1]), # "ARCADE" button
  Screen.IN_GAME: lambda: None, 
  Screen.ARCADE: lambda: io.click_mouse(450, 875, delays=[0.01,0,0.0]), # "Next Games" button
  Screen.ARCADE_GOOBER_BOTTOM: lambda: io.click_mouse(420, 770, delays=[0.1,0,0]), # "Goober's Lab" logo
  Screen.ARCADE_GOOBER_SELECTED: lambda: io.click_mouse(1200, 900, delays=[0.1,0,0]), # "PLAY GAME" button
  Screen.GREAT_PLAY: lambda: io.click_mouse(800, 1100, delays=[0.1,0,0]) # "Collect" button
}

SCREEN_TYPES = {
  Screen.OUT_OF_MOVES: [275, 600, [187, 38, 26]], # red "O" in "OOPS"
  Screen.LEVEL_COMPLETE: [1330, 1050, [24, 62, 12]], # green BR corner of chalk board
  Screen.OUT_OF_TIME: [273, 606, [100, 73, 40]], # brown background shelf
  Screen.HOME: [1308, 1195, [233, 51, 35]], # Goober's red shoe
  Screen.GAME_OVER: [760, 67, [125, 251, 76]], # bright green arcade cabinet bezel
  Screen.IN_GAME: [1530, 977, [103, 103, 103]], # grey lid of brain jar
  Screen.ARCADE: [400, 1100, [179, 119, 185]], # magenta arcade cabinet
  Screen.ARCADE_GOOBER_BOTTOM: [137, 832, [246, 203, 69]], # yellow BL logo corner
  Screen.ARCADE_GOOBER_SELECTED: [1175, 645, [25, 64, 14]], # green chalkboard BC
  Screen.GREAT_PLAY: [470, 800, [173, 67, 58]], # Red gift box lid
}

ATOM_COLORS = {
  Atom.BLUE: [0,27,227],
  Atom.GREEN: [108,197,59],
  Atom.CYAN: [108,235,237],
  Atom.PURPLE: [139,33,156],
  Atom.PINK: [234,59,183],
  Atom.RED: [234,54,40],
  Atom.ORANGE: [239,134,51],
  Atom.YELLOW: [249,240,82],
  Atom.BLANK: [151,151,151] # TODO: check correctness
}

DOT_COLORS = {
  Atom.BLUE: [230, 167, 75],        # light orange
  Atom.YELLOW: [120, 94, 233],      # light purple
  Atom.CYAN: [227, 132, 69],        # light orange
  Atom.RED: [134, 237, 87],         # bright green
  Atom.GREEN: [222, 69, 92],        # light red
  Atom.PINK: [211, 238, 90],        # yellow-green
  Atom.PURPLE: [242, 240, 92],      # yellow
  Atom.ORANGE: [132, 238, 239],     # cyan
  Atom.BLANK: [151, 151, 151],      # blank (no dot)
}

ATOM_INFO = {
  Atom.BLUE: {'ansi': 21, 'dot_ansi': 215}, 
  Atom.GREEN: {'ansi': 76, 'dot_ansi': 204}, 
  Atom.CYAN: {'ansi': 51, 'dot_ansi': 209}, 
  Atom.PURPLE: {'ansi': 127, 'dot_ansi': 227}, 
  Atom.PINK: {'ansi': 200, 'dot_ansi': 191}, 
  Atom.RED: {'ansi': 196, 'dot_ansi': 83}, 
  Atom.ORANGE: {'ansi': 214, 'dot_ansi': 87}, 
  Atom.YELLOW: {'ansi': 226, 'dot_ansi': 105}, 
  Atom.BLANK: {'ansi': 145, 'dot_ansi': 145}, 
}

STYLE_INFO = {
  Style.PLAIN: {'char': ' '},
  Style.DOT: {'char': '●'}, # '●' or '•'
  Style.STAR: {'char': '✻'},
}

####################################################################################
# ATOM PARSING, SAVING, DISPLAYING METHODS
####################################################################################

def get_atom_color(atom_img):
  atom_color = io.crop_pixel(58, 63, atom_img)[0,0]
  best_color_idx, dist = utils.IOHandler.get_best_color(atom_color, ATOM_COLORS.values())
  best_color_enum = list(ATOM_COLORS.keys())[best_color_idx]
  if dist > 10:
    print(f"WARNING: atom color kinda far (>10): dist({atom_color}, {best_color_enum}) = {dist}")
    # return None
  return best_color_enum

# Check if the atom is a special 5 "bonus" atom, based on presence of star color
def has_star(atom_img):
  TARGET_COLOR = (255, 254, 209)
  MIN_COUNT, MAX_COUNT = 370, 1150
  mask = np.all(atom_img == TARGET_COLOR, axis=2)
  count = np.count_nonzero(mask)
  if MIN_COUNT <= count <= MAX_COUNT:
    return True
  if count > 0:
     print("WARNING, star pixels outside of range:", count)
  return False

# Check if the atom is a special 4 "unstable" atom, based on presence of dot color
def has_dot(atom_img, atom_enum):
  if atom_enum == Atom.BLANK:
    return False
  target_color = DOT_COLORS[atom_enum]
  MAX_DELTA = 5
  MIN_COUNT, MAX_COUNT = 56, 350

  difference = np.abs(atom_img - target_color)
  distance = np.sqrt(np.sum(np.square(difference), axis=2))
  mask = distance <= MAX_DELTA

  count = np.sum(mask)
  if MIN_COUNT <= count <= MAX_COUNT:
    return True
  if count > 0:
    print("WARNING: has dot count out of range: ", count)
  return False

def get_atom_style(atom_img, atom_enum):
  if (has_star(atom_img)):
    return Style.STAR
  if (has_dot(atom_img, atom_enum)):
    return Style.DOT
  return Style.PLAIN

# Check if the atom is selected, by looking for green border pixel
def is_atom_selected(atom_img):
  return io.does_pixel_equal(13, 13, (125, 250, 76), delta=10, image=atom_img)

def get_atom_info(atom_img):
  atom_enum = get_atom_color(atom_img)
  atom_style = get_atom_style(atom_img, atom_enum)
  atom_selected = is_atom_selected(atom_img)
  return {'color': atom_enum, 'style': atom_style, 'selected': atom_selected}

def parse_board_from_game_img(game_img):
  bx, by, aw, ah = 37, 227, 118, 118
  board = []
  for j in range(8):
    board.append([])
    ay = by + ah*j
    for i in range(8):
      ax = bx + aw*i
      atom_img = io.crop_portion(ax, ay, aw, ah, game_img)
      atom_info = get_atom_info(atom_img)
      board[j].append(atom_info)
  return board

def save_atoms_in_game_img(game_img):
  bx, by, aw, ah = 37, 227, 118, 118
  for j in range(8):
    ay = by + ah*j
    for i in range(8):
      ax = bx + aw*i
      atom_img = io.crop_portion(ax, ay, aw, ah, game_img)
      atom_info = get_atom_info(atom_img)
      color_str = atom_info['color'].value
      selected_str = 'selected' if atom_info['selected'] else 'unselected'
      style_str = atom_info['style'].value
      io.save_image(atom_img, f"./atoms/{color_str}_{selected_str}_{style_str}_{i}_{j}.png")

def save_all_atom_images():
  game_imgs = io.load_images_from_directory('./atoms_to_save')
  for img_name, game_img in game_imgs.items():
    save_atoms_in_game_img(game_img)

def get_atom_str(atom, p=1):
  atom_enum = atom['color']
  atom_style = atom['style']
  atom_selected = atom['selected']
  atom_char = STYLE_INFO[atom_style]['char']
  atom_code = ATOM_INFO[atom_enum]['ansi']
  atom_style_str = (u"\u001b[48;5;" + str(atom_code) + "m")
  char_style = ''
  if atom_style == Style.DOT:
    char_code = ATOM_INFO[atom_enum]['dot_ansi']
    char_style = (u"\u001b[38;5;" + str(char_code) + "m")
  elif atom_style == Style.STAR:
    char_style = (u"\u001b[38;5;" + str(250) + "m")
  selected_style = ''
  selected_char = ' '
  if atom_selected:
    selected_style = (u'\u001b[1m' + u"\u001b[38;5;" + str(0) + "m")
    selected_char = "│"
  return selected_style + atom_style_str + (selected_char)*p + (char_style) + atom_char + (selected_style) + (selected_char) + u"\u001b[0m"

def print_board(board_atoms, p=1):
  M, N = len(board_atoms), len(board_atoms[0])
  output = []
  output.append((" "*p +" ").join([f"{x:>2}" for x in [''] + list(range(N))]))
  for i, row in enumerate(board_atoms):
      line = f"{i:>2} "
      for atom in row:
          line += get_atom_str(atom, p=p) + " "
      output.append(f"{line}")
  output = '\n'.join(output)
  print(output)
  return output

####################################################################################
# SOLVING METHODS
####################################################################################

def find_valid_swaps(board, level):
  DIR_OFFSETS = [(1, 0), (0, 1)]
  valid_swaps = set()
  for x1 in range(8):
    for y1 in range(8):
      for dx, dy in DIR_OFFSETS:
        x2, y2 = x1 + dx, y1 + dy
        if x2 < 0 or x2 >= 8 or y2 < 0 or y2 >= 8:
          continue
        pair = ((x1,y1), (x2,y2))
        # TODO: ignore swap with blanks or two of the same color
        new_board, est_score = simulate_swap(board, pair, level)
        if est_score > 0:
          valid_swaps.add((pair, est_score))
  valid_swaps = sorted(valid_swaps, key=lambda x: -x[1])
  return valid_swaps

def est_plain_RX_pts(length, level, indirects):
  RX_points = (length - 2) * (5 + 5*level)
  indir_points = RX_points * (1 + indirects*0.2)
  return int(indir_points)

# Pops matches on the board once (e.g. pop matches, pop bonuses, place bonuses, apply gravity)
def pop_board_once(board, level, indirects):
  new_board = [[cell.copy() for cell in row] for row in board]
  # print("PRE-POP ONCE")
  # print_board(new_board)
  groups = get_board_groups(new_board)
  # print("GROUPS:", len(groups))
  bonus_to_place = []
  bonus_to_redeem = []
  est_points = 0
  for group in groups:
    size = len(group)
    color = group[0]['type']['color']
    # Clear all the matches and find what bonuses we'll place and redeem
    est_points += est_plain_RX_pts(len(group), level, indirects)
    # print(f"EST POINTS (after group R{len(group)}):", est_points)
    indirects += 1
    for cell in group:
      i, j = cell['i'], cell['j']
      style = new_board[i][j]['style']
      if style in [Style.STAR, Style.DOT]:
        bonus_to_redeem.append(copy.deepcopy(cell))
      new_board[i][j]['color'] = Atom.BLANK
      new_board[i][j]['style'] = Style.PLAIN
    # Add to the list of bonus atoms that we'll create later
    # TODO: make sure we don't place two bonuses in same spot (e.g. X4)
    if size > 3:
      style = Style.DOT if size == 4 else Style.STAR
      # If one of the atoms in the group is the one we swapped (cell selected), we'll put the bonus atom there, else we'll slap it in the middle
      new_bonus_coords = [i for i in range(size) if new_board[cell['i']][cell['j']]['selected']]
      if len(new_bonus_coords) == 0:
        new_bonus_coords = [(size - 1) // 2]
      bonus_cell = group[new_bonus_coords[0]]
      bonus_to_place.append(bonus_cell)
  # print("REMOVED GROUPS")
  # print_board(new_board)

  bonus_to_redeem = sorted(bonus_to_redeem, key=lambda x: x['type']['style'] == Style.DOT, reverse=True) # Do the dots first to be conservative
  # Redeems bonuses (e.g. apply dots (remove 5) and stars (remove one color))
  redeemed_coords = set()
  redeemed_star_colors = set()
  depth = 0
  MAX_DEPTH = 10
  while len(bonus_to_redeem) != 0 and depth < MAX_DEPTH:
    # print('BONUSES TO REDEEM:', len(bonus_to_redeem), bonus_to_redeem)
    # print("REDEEMED COORDS", redeemed_coords)
    new_bonus_to_redeem = []
    for bonus_cell in bonus_to_redeem:
      i, j = bonus_cell['i'], bonus_cell['j']
      if (i, j) in redeemed_coords:
        print(f"WARNING: Already redeemed bonus ({i}, {j}), skipping:", bonus_cell)
        continue
      bonus_color, bonus_style = bonus_cell['type']['color'], bonus_cell['type']['style']
      if bonus_style == Style.STAR:
        if bonus_color in redeemed_star_colors:
          print(f"WARNING: already redeemed star of color {bonus_color}. Skipping.")
          redeemed_coords.add((i, j))
          continue
        # TODO: Factor in kz bonus chance?
        new_board, new_points, new_redeems = redeem_star_atom(new_board, bonus_color, level, kinzcash_chance=False)
        new_bonus_to_redeem += new_redeems
        est_points += new_points
        # print("EST POINTS AFTER STAR:", est_points, new_points)
        redeemed_star_colors.add(bonus_color)
      # NOTE: not sure if I should redeem dots since they have a random factor
      elif bonus_style == Style.DOT:
        new_board, new_points, new_redeems = redeem_dot_atom(new_board, level)
        new_bonus_to_redeem += new_redeems
        est_points += new_points
      redeemed_coords.add((i, j))
    bonus_to_redeem = new_bonus_to_redeem
    # print("REDEEMED BONUSES")
    # print_board(new_board)
    depth += 1
  if depth >= MAX_DEPTH:
    print("WARNING: reached MAX_DEPTH, probably an infinite loop issue")

  # TODO: could make sure that two bonuses not in same place (e.g. X4)
  # print('BONUSES TO PLACE:', len(bonus_to_place), bonus_to_place)
  # Places bonus cells
  for bonus_cell in bonus_to_place:
    i, j = bonus_cell['i'], bonus_cell['j']
    new_board[i][j]['color'] = bonus_cell['type']['color']
    new_board[i][j]['style'] = bonus_cell['type']['style']
  # print("PLACED BONUSES")
  # print_board(new_board)

  new_board = apply_gravity(new_board)
  # print("APPLIED GRAVITY")
  # print_board(new_board)
  return new_board, est_points, indirects

def boards_equal(board1, board2):
  M, N = len(board1), len(board1[0])
  for i in range(M):
    for j in range(N):
      cell1 = board1[i][j]
      cell2 = board2[i][j]
      if cell1['color'] != cell2['color'] or cell1['style'] != cell2['style']:
        return False
  return True

def simulate_swap(board, pair, level):
  # print("PRE-SWAP BOARD")
  # print_board(board)
  coord_1, coord_2 = pair
  i1, j1 = coord_1
  i2, j2 = coord_2
  new_board = [[cell.copy() for cell in row] for row in board]
  new_board[i1][j1], new_board[i2][j2] = new_board[i2][j2], new_board[i1][j1]
  # print("SWAPPED ATOMS")
  # print_board(new_board)

  MAX_LOOPS = 10
  indirects = 0
  est_points = 0
  loops = 0

  old_board = new_board
  
  while loops < MAX_LOOPS and (loops == 0 or not boards_equal(old_board, new_board)):
    old_board = new_board
    new_board, new_points, new_indirects = pop_board_once(new_board, level, indirects)
    est_points += new_points
    indirects = new_indirects
    loops += 1

  if loops >= MAX_LOOPS:
    print(f"WARNING loops > MAX_LOOPS ({loops}), maybe infinite loop")
  
  return new_board, est_points
  

# Apply gravity to the board so all the blanks go to the top
def apply_gravity(board):
  M, N = len(board), len(board[0])
  new_board = [[{'color': Atom.BLANK, 'style': Style.PLAIN, 'selected': cell['selected']} for cell in row] for row in board]
  column_atoms = [[board[i][j] for i in range(M) if board[i][j]['color'] != Atom.BLANK] for j in range(N)]
  for j, col in enumerate(column_atoms):
    for i, cell in enumerate(col):
      ni = i + (N - len(col))
      new_board[ni][j]['color'] = cell['color']
      new_board[ni][j]['style'] = cell['style']
  return new_board

def random_atom_color(level):
  level_colors = [x for x in list(Atom) if x != Atom.BLANK and not (x == Atom.CYAN and level < 9 or x == Atom.ORANGE and level < 5)]
  return random.choice(level_colors)

def random_atom_style(level):
  star_chance = 0 if level < 5 else 0.01
  return random.choices([Style.PLAIN, Style.STAR], weights=[1 - star_chance, star_chance], k=1)[0]

# Fill in blanks in the board randomly
def fill_blanks_randomly(board, level):
  new_board = [[cell.copy() for cell in row] for row in board]
  for row in new_board:
    for cell in row:
      if cell['color'] == Atom.BLANK:
        cell['color'] = random_atom_color(level)
        cell['style'] = random_atom_style(level)
  return new_board

def redeem_star_atom(board, target_color, level, kinzcash_chance):
  use_kinzcash_bonus = kinzcash_chance and random.random() < 0.5
  new_board = [[cell.copy() for cell in row] for row in board]

  if use_kinzcash_bonus:
    est_points = 5 + 3*level # r5(kb) = (5 + 3L)
    return new_board, est_points, []
  
  bonus_to_redeem = []
  extra = 0
  for i, row in enumerate(new_board):
    for j, cell in enumerate(row):
      style, color = cell['style'], cell['color']
      # TODO: make sure not redeeming same thing multiple times?
      if color == target_color:
        # Don't redeem star twice, since it doesn't count twice
        if style in [Style.DOT]:
          bonus_to_redeem.append({'type': cell.copy(), 'i': i, 'j': j})
        cell['color'] = Atom.BLANK
        cell['style'] = Style.PLAIN
        extra += 1
  est_points = 3*level*extra + 9*level + 5*extra + 15 # r5(+E) = (3LE + 9L + 5E + 15)
  return new_board, est_points, bonus_to_redeem

def redeem_dot_atom(board, level):
  K = 5
  M, N = len(board), len(board[0])
  new_board = [[cell.copy() for cell in row] for row in board]
  non_blank_indices = [(i, j) for j in range(M) for i in range(N) if new_board[i][j]['color'] != Atom.BLANK]
  chosen_indices = random.sample(non_blank_indices, K)
  bonus_to_redeem = []
  for i, j in chosen_indices:
    cell = new_board[i][j]
    color, style = cell['color'], cell['style']
    if style in [Style.DOT, Style.STAR]:
      bonus_to_redeem.append({'type': cell.copy(), 'i': i, 'j': j})
    cell['color'] = Atom.BLANK
    cell['style'] = Style.PLAIN
  est_points = 25*level
  return new_board, est_points, bonus_to_redeem

def get_rowcol_groups(rowcol):
    groups = []
    group = []
    for idx in range(len(rowcol)):
        cell = rowcol[idx]
        if len(group) == 0 or group[-1]['type']['color'] == cell['color']:
             group.append({'type': cell, 'idx': idx})
        else:
            groups.append(group)
            group = []
            group.append({'type': cell, 'idx': idx})
    if len(group) > 0:
        groups.append(group)
    return groups

def get_board_groups(board):
  groups = get_all_board_groups(board)
  valid_groups = [g for g in groups if len(g) >= 3 and g[0]['type']['color'] != Atom.BLANK]
  valid_groups = sorted(valid_groups, key = lambda g: -len(g)) # Sorted in desc order, for a conservative est.
  return valid_groups

def get_all_board_groups(board):
    board = [[cell.copy() for cell in row] for row in board]
    M, N = len(board), len(board[0])
    groups = []
    for i in range(M):
        row_groups = get_rowcol_groups(board[i])
        groups += [[{'type': c['type'], 'i': i, 'j': c['idx'], 'dir': 'H'} for c in g] for g in row_groups]
    for j in range(N):
        col_groups = get_rowcol_groups([board[i][j] for i in range(M)])
        groups += [[{'type': c['type'], 'i': c['idx'], 'j': j, 'dir': 'V'} for c in g] for g in col_groups]
    return groups

def calc_time_bonus(level, time_left):
   return level * time_left

####################################################################################
# HELPERS
####################################################################################
# TODO: move ansi helpers to utils
def calc_ansi_map():
  ansi_img = io.load_image('./ansi-colors.png')
  ansi_map = {}
  ox, oy = 21, 23
  w, h = 50, 37
  for i in range(16):
      for j in range(16):
          idx = j + 16*i
          x = ox + w*j
          y = oy + h*i
          color = io.crop_pixel(x, y, ansi_img)
          ansi_map[idx] = list(color[0,0])
  return ansi_map

# TODO: move ansi stuff to utils?
def find_nearest_ansi(target, ansi_map):
    best_idx, best_dist = utils.IOHandler.get_best_color(target, ansi_map.values())
    best_ansi = list(ansi_map.keys())[best_idx]
    return best_ansi

# TODO: make quit key consistent
def on_press(key):
  global kill_program
  global listener
  if key == kb.KeyCode.from_char('1'):
    print("'1' pressed to quit")
    listener.stop()
    kill_program = True

def is_screen_type(screen_type, image=None):
    x, y, color = SCREEN_TYPES[screen_type]
    return io.is_given_screen(x, y, color, delta=10, image=image)

# TODO: could move to utils
def calc_screen_types(image=None):
    return {screen_type for screen_type in SCREEN_TYPES.keys() if is_screen_type(screen_type, image)}

def get_into_game():
  global kill_program, LEVEL
  # Get to the main play screen
  screen_types = set()
  while True:
    if kill_program:
      break
    io.move_mouse(0, 0, delay=0.1)
    game_img = io.capture_screen()
    last_screen_types = screen_types
    screen_types = calc_screen_types(game_img)
    print(screen_types)

    if Screen.GREAT_PLAY in screen_types:
      SCREEN_ACTIONS[Screen.GREAT_PLAY]()
    elif Screen.GAME_OVER in screen_types:
      SCREEN_ACTIONS[Screen.GAME_OVER]()
    elif Screen.HOME in screen_types:
      SCREEN_ACTIONS[Screen.HOME]()
      if Screen.HOME not in last_screen_types:
        LEVEL = 1
        print(f"LEVEL {LEVEL}")
    elif Screen.OUT_OF_TIME in screen_types:
      SCREEN_ACTIONS[Screen.OUT_OF_TIME]()
    elif Screen.OUT_OF_MOVES in screen_types:
      SCREEN_ACTIONS[Screen.OUT_OF_MOVES]()
    elif Screen.LEVEL_COMPLETE in screen_types:
      SCREEN_ACTIONS[Screen.LEVEL_COMPLETE]()
      if Screen.LEVEL_COMPLETE not in last_screen_types:
        LEVEL += 1
        print(f"LEVEL {LEVEL}")
    elif Screen.IN_GAME in screen_types:
      board = parse_board_from_game_img(game_img)
      M, N = len(board), len(board[0])
      selected_atoms = [(i, j) for i in range(M) for j in range(N) if board[i][j]['selected']]
      blank_atoms = [(i, j) for i in range(M) for j in range(N) if board[i][j]['color'] == Atom.BLANK]
      if len(blank_atoms) > 0:
        continue
      if len(selected_atoms) == 1:
        for i, j in selected_atoms:
          click_atom(i, j)
      elif len(selected_atoms) == 0:
        return
    elif Screen.ARCADE in screen_types:
      if Screen.ARCADE_GOOBER_SELECTED in screen_types:
        SCREEN_ACTIONS[Screen.ARCADE_GOOBER_SELECTED]()
      elif Screen.ARCADE_GOOBER_BOTTOM in screen_types:
        SCREEN_ACTIONS[Screen.ARCADE_GOOBER_BOTTOM]()
      else:
        SCREEN_ACTIONS[Screen.ARCADE]()

    time.sleep(0.2)

def click_atom(i, j, delays=[0.3, 0.1, 0.3]):
  bx, by, aw, ah = 37, 227, 118, 118
  ay = by + ah*i
  ax = bx + aw*j
  io.click_mouse(ax + aw//2, ay + ah//2, delays=delays)

ANSI_MAP = {0: [0, 0, 0], 1: [236, 96, 115], 2: [202, 230, 151], 3: [247, 204, 122], 4: [137, 170, 249], 5: [191, 149, 228], 6: [155, 219, 251], 7: [255, 255, 255], 8: [74, 74, 74], 9: [236, 96, 115], 10: [202, 230, 151], 11: [247, 204, 122], 12: [137, 170, 249], 13: [191, 149, 228], 14: [155, 219, 251], 15: [255, 255, 255], 16: [0, 0, 0], 17: [0, 1, 48], 18: [0, 6, 97], 19: [0, 14, 147], 20: [0, 22, 196], 21: [0, 30, 245], 22: [18, 50, 8], 23: [18, 50, 51], 24: [16, 51, 98], 25: [14, 53, 147], 26: [9, 56, 196], 27: [2, 60, 245], 28: [42, 99, 25], 29: [42, 99, 56], 30: [41, 100, 101], 31: [40, 101, 149], 32: [39, 102, 197], 33: [36, 104, 246], 34: [67, 150, 42], 35: [67, 150, 65], 36: [67, 150, 106], 37: [66, 151, 152], 38: [65, 152, 199], 39: [64, 153, 247], 40: [92, 200, 59], 41: [92, 200, 77], 42: [92, 200, 113], 43: [91, 200, 156], 44: [91, 201, 202], 45: [90, 202, 250], 46: [117, 249, 76], 47: [117, 249, 90], 48: [117, 250, 122], 49: [117, 250, 162], 50: [116, 250, 207], 51: [115, 251, 253], 52: [46, 4, 2], 53: [46, 5, 49], 54: [45, 10, 98], 55: [45, 17, 147], 56: [43, 24, 196], 57: [41, 32, 245], 58: [51, 51, 10], 59: [51, 51, 51], 60: [50, 52, 99], 61: [50, 54, 147], 62: [48, 57, 196], 63: [46, 60, 245], 64: [64, 100, 26], 65: [64, 100, 57], 66: [64, 101, 101], 67: [63, 102, 149], 68: [62, 103, 197], 69: [61, 105, 246], 70: [82, 150, 42], 71: [82, 150, 66], 72: [82, 150, 106], 73: [81, 151, 152], 74: [81, 152, 199], 75: [80, 153, 247], 76: [103, 200, 59], 77: [103, 200, 77], 78: [103, 200, 113], 79: [103, 200, 156], 80: [102, 201, 203], 81: [101, 202, 250], 82: [125, 249, 76], 83: [125, 249, 90], 84: [125, 250, 122], 85: [125, 250, 162], 86: [124, 250, 207], 87: [124, 251, 253], 88: [93, 14, 7], 89: [93, 15, 50], 90: [93, 18, 98], 91: [93, 23, 147], 92: [92, 29, 196], 93: [91, 36, 245], 94: [95, 54, 15], 95: [95, 54, 52], 96: [95, 55, 99], 97: [95, 57, 148], 98: [94, 59, 196], 99: [93, 63, 245], 100: [102, 101, 28], 101: [102, 101, 58], 102: [102, 102, 102], 103: [102, 103, 149], 104: [101, 104, 198], 105: [100, 106, 246], 106: [113, 151, 44], 107: [113, 151, 67], 108: [113, 151, 107], 109: [113, 152, 152], 110: [112, 153, 200], 111: [112, 154, 248], 112: [128, 201, 61], 113: [128, 201, 78], 114: [128, 201, 114], 115: [128, 201, 157], 116: [127, 202, 203], 117: [127, 203, 250], 118: [146, 250, 77], 119: [146, 250, 91], 120: [146, 251, 122], 121: [146, 251, 163], 122: [146, 251, 207], 123: [145, 252, 253], 124: [140, 26, 17], 125: [140, 26, 53], 126: [140, 28, 100], 127: [140, 32, 148], 128: [139, 36, 196], 129: [139, 42, 245], 130: [141, 58, 22], 131: [141, 58, 55], 132: [141, 59, 100], 133: [141, 60, 148], 134: [140, 63, 197], 135: [140, 66, 246], 136: [146, 103, 33], 137: [146, 103, 60], 138: [146, 104, 103], 139: [146, 105, 150], 140: [146, 106, 198], 141: [145, 108, 246], 142: [153, 152, 47], 143: [153, 152, 68], 144: [153, 152, 108], 145: [153, 153, 153], 146: [153, 154, 200], 147: [152, 155, 248], 148: [164, 202, 63], 149: [164, 202, 80], 150: [164, 202, 115], 151: [164, 202, 157], 152: [164, 203, 203], 153: [163, 204, 250], 154: [177, 250, 79], 155: [177, 250, 92], 156: [177, 251, 123], 157: [177, 251, 163], 158: [177, 251, 208], 159: [176, 252, 254], 160: [187, 39, 26], 161: [187, 39, 57], 162: [187, 40, 101], 163: [187, 43, 149], 164: [187, 46, 197], 165: [187, 51, 246], 166: [188, 63, 30], 167: [188, 64, 58], 168: [188, 65, 102], 169: [188, 66, 149], 170: [188, 68, 198], 171: [188, 71, 246], 172: [191, 106, 39], 173: [191, 106, 63], 174: [191, 107, 105], 175: [191, 108, 151], 176: [191, 109, 199], 177: [191, 111, 247], 178: [196, 154, 51], 179: [196, 154, 71], 180: [196, 154, 109], 181: [196, 155, 154], 182: [196, 156, 201], 183: [196, 157, 249], 184: [204, 203, 66], 185: [204, 203, 82], 186: [204, 203, 116], 187: [204, 203, 158], 188: [204, 204, 204], 189: [203, 205, 251], 190: [214, 252, 81], 191: [214, 252, 94], 192: [214, 253, 125], 193: [214, 253, 164], 194: [214, 253, 208], 195: [214, 254, 254], 196: [234, 50, 35], 197: [234, 51, 61], 198: [234, 52, 104], 199: [234, 54, 150], 200: [234, 57, 198], 201: [234, 60, 247], 202: [235, 71, 38], 203: [235, 72, 63], 204: [235, 72, 105], 205: [235, 74, 151], 206: [235, 76, 199], 207: [235, 78, 247], 208: [237, 111, 45], 209: [237, 111, 67], 210: [237, 112, 107], 211: [237, 113, 153], 212: [237, 114, 200], 213: [237, 115, 248], 214: [242, 157, 56], 215: [241, 157, 75], 216: [241, 157, 112], 217: [241, 158, 156], 218: [241, 159, 202], 219: [241, 160, 249], 220: [247, 205, 70], 221: [247, 205, 85], 222: [247, 205, 118], 223: [247, 205, 160], 224: [247, 206, 205], 225: [247, 207, 252], 226: [255, 253, 84], 227: [255, 253, 97], 228: [255, 254, 126], 229: [255, 254, 166], 230: [255, 254, 209], 231: [255, 255, 255], 232: [0, 0, 0], 233: [11, 11, 11], 234: [22, 22, 22], 235: [33, 33, 33], 236: [44, 44, 44], 237: [55, 55, 55], 238: [67, 67, 67], 239: [78, 78, 78], 240: [89, 89, 89], 241: [100, 100, 100], 242: [111, 111, 111], 243: [122, 122, 122], 244: [133, 133, 133], 245: [144, 144, 144], 246: [155, 155, 155], 247: [166, 166, 166], 248: [177, 177, 177], 249: [188, 188, 188], 250: [200, 200, 200], 251: [211, 211, 211], 252: [222, 222, 222], 253: [233, 233, 233], 254: [244, 244, 244], 255: [255, 255, 255]}

In [ ]:
kill_program = False
listener = kb.Listener(on_press=on_press)
listener.start()

LEVEL = 1
# Run game loop
io.click_mouse(0, 0, delays=[0.1,0,0.0]) # Focus webkinz
while True:
  if kill_program:
      break
  get_into_game()
  game_img = io.capture_screen()

  board = parse_board_from_game_img(game_img)
  valid_swaps = find_valid_swaps(board, level=LEVEL)
  valid_swaps = sorted(valid_swaps, key=lambda x: (-x[1], min(x[0][0][0], x[0][1][0]), max(x[0][0][0], x[0][1][0])))

  if len(valid_swaps) == 0:
    print("WARNING: Think we're out of moves... waiting")
    time.sleep(0.1)
    continue

  coord1, coord2 = valid_swaps[0][0]
  click_atom(*coord1, delays=[0.1,0,0])
  click_atom(*coord2, delays=[0.1,0,0])

listener.stop()

# Notes
## Picking a good move
- Combos are better than individual (e.g. chain together after atoms fall, etc... indirect matches)
- Prioritize not dying 1st. Then chaining as many together as we can and creating/redeeming bonuses.
- To decide the best swap, need to simulate what happens & also look a couple turns ahead (e.g. lining up a 5 or chaining with gravity)
  - Some actions are random (e.g. redeeming some of them sometimes give kinzcash bonus, stuff that drops from above is random (seems an even spread), redeeming a 4 seems random which 5 picked)
    - Could simulate once with random values, simulate multiple times and average / min-max, or calculate expected value somehow. Could also just ignore random elements (e.g. not fill in from top, pretend always a kinzcash bonus)